In [3]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc
from time import sleep
import pandas as pd
import sqlite3
import time
import re


REMOVE_ADS_SCRIPT = """
var ads = document.querySelectorAll('[id^="google_ads"], [class*="ads"], [class*="sponsored"]');
ads.forEach(ad => ad.remove());
"""


def start_google_chrome(url, screen_width=2200, screen_height=1900):
    """
    Undetected Chrome を起動し、指定URLにアクセス
    - ウィンドウを左半分に配置
    - 広告対策オプションを追加
    :param url: 開くURL
    :param screen_width: 画面の横幅 (デフォルト: 2200)
    :param screen_height: 画面の高さ (デフォルト: 1900)
    :return: WebDriver インスタンス
    """

    # Chromeオプションを設定
    options = uc.ChromeOptions()
    options.add_argument("--no-sandbox")  # セキュリティサンドボックス無効化
    options.add_argument("--disable-blink-features=AutomationControlled")  # ボット検出回避
    options.add_argument("--disable-extensions")  # 拡張機能無効化
    options.add_argument("--disable-popup-blocking")  # ポップアップブロック無効化
    options.add_argument("--disable-gpu")  # GPUレンダリング無効化
    options.add_argument("--start-maximized")  # 最大化
    options.add_argument("--disable-infobars")  # "Chromeは自動テストソフトウェアによって制御されています"を非表示
    options.add_argument("--disable-notifications")  # 通知をブロック
    options.add_argument("--remote-debugging-port=9222")  # デバッグ用ポート

    # Chromeドライバーの起動
    driver = uc.Chrome(options=options)

    # 画面の左半分にウィンドウを配置
    driver.set_window_position(0, 0)  # 左上 (0,0) に配置
    driver.set_window_size(screen_width // 2, screen_height)  # 画面の半分の幅に設定

    # 指定URLを開く
    driver.get(url)
    print("-" * 50)
    print(url)
    driver.implicitly_wait(30)  # 最大30秒待機
    time.sleep(1)  # 読み込み待機

    return driver  # WebDriver インスタンスを返す



def click_date_link(driver, DAYS_AGO, MAX_RETRIES=3):
    """
    指定したDAYS_AGOの日付リンクをクリックし、ページ遷移を確認する
    :param driver: Selenium WebDriver
    :param DAYS_AGO: 過去の日数（1=最新の日付）
    :param MAX_RETRIES: 最大リトライ回数
    :return: 成功した場合はTrue、失敗した場合はFalse
    """
    retries = 0
    current_url = driver.current_url

    try:
        wait = WebDriverWait(driver, 10)
        date_links = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".table-data-cell a")))

        if DAYS_AGO > len(date_links):
            print(f"エラー: DAYS_AGO={DAYS_AGO} に対応するリンクがありません。")
            return False

        target_link = date_links[DAYS_AGO - 1]
        print(f"クリック対象リンク: {target_link.text}")

        while retries < MAX_RETRIES:
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'auto', block: 'center'});", target_link)
            time.sleep(2)  # スクロール後の待機
            wait.until(EC.element_to_be_clickable(target_link))
            time.sleep(2)

            target_link.click()

            wait.until(EC.url_changes(current_url))
            time.sleep(2)

            if "google_vignette" in driver.current_url:
                retries += 1
                print(f"⚠ Google広告が表示されました。リトライします ({retries}/{MAX_RETRIES})")
                driver.back()  # 広告ページから戻る
                time.sleep(2)
                continue  # リトライ

            print(f"遷移成功: {driver.current_url}")
            date, hall_name = extract_date_hall(driver.current_url)
            return date, hall_name

    except Exception as e:
        print(f"エラー発生: {e}")
    
    print("クリック失敗: リトライ回数超過")
    return False  # 失敗


def extract_date_hall(url):
    """
    URL から日付 (YYYY-MM-DD) とホール名を抽出
    :param url: 解析するURL
    :return: (date, hall_name) のタプル
    """
    pattern = r"https://ana-slo\.com/(\d{4})-(\d{1,2})-(\d{1,2})-(.+)-data/"
    match = re.search(pattern, url)

    if match:
        year, month, day, hall_name = match.groups()
        date = f"{year}-{int(month):02d}-{int(day):02d}"  # 月日をゼロ埋め
        hall_name = hall_name.replace("-", "_").upper()  # "-" をスペースに置き換えて大文字に
        print(f"日付: {date}")
        print(f"ホール名: {hall_name}")
        return date, hall_name
    else:
        return None, None  # マッチしない場合

    
def click_machine_by_name(driver, search_machine):
    """
    指定した機種名を検索し、該当する要素をクリック
    :param driver: Selenium WebDriver
    :param search_machine: 検索する機種名
    :return: クリック成功: True / 失敗: False
    """
    try:
        sections = driver.find_elements(By.CLASS_NAME, "unit_get_medals_table a")
        for i, section in enumerate(sections):
            if search_machine in section.text:
                machine_name = section.text
                section_num = i
                # print(f"{i}: {machine_name}")
                driver.execute_script("arguments[0].scrollIntoView({behavior: 'auto', block: 'nearest'});", section)
                sleep(3)
                section.click()
                print(f"クリック成功: {machine_name}")
                return section_num
            
        print("指定の機種が見つかりませんでした。")
        return False
    
    except Exception as e:
        print(f"❌ エラー発生: {e}")
        return False


def extract_and_save_model_data(driver, section_num, hall_name, date):
    """
    指定した機種のデータを抽出し、CSVに保存する
    :param driver: Selenium WebDriver
    :param section_num: セクション番号 (該当機種のインデックス)
    :param hall_name: ホール名
    :param date: 日付 (yyyy-mm-dd)
    :return: 成功時 True, 失敗時 False
    """
    try:
        # 機種名を取得
        model_name = driver.find_element(By.ID, f"section{section_num}").text
        
        # データ行を取得
        rows = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, f"#tab01_{section_num} div table tbody tr"))
        )
        
        if not rows:
            print(f"⚠️ データが見つかりません: {model_name}")
            return False

        print(f"{model_name}: {len(rows)} units 取得開始")
        print("取得中...")

        # ヘッダー取得
        header_cells = rows[0].find_elements(By.TAG_NAME, "th")
        columns = [cell.text for cell in header_cells]

        # データ取得
        data = []
        for row in rows[1:-1]:  # 最終行を除外
            cells = row.find_elements(By.TAG_NAME, "td")
            data.append([cell.text for cell in cells])

        print(f"{model_name}: {len(data)} units 取得完了")

        # DataFrameを作成しCSVに保存
        df = pd.DataFrame(data, columns=columns)
        file_name = f"../csv/{hall_name}_{model_name}_{date}.csv"
        df.to_csv(file_name, index=False, encoding="utf-8-sig")
        
        print(rows[1].text)

        print(f"✅ データ保存完了: {file_name}")
        return True

    except Exception as e:
        print(f"❌ エラー発生: {e}")
        return False



URL = f"https://ana-slo.com/"
DATA = "ホールデータ"
PREF = "東京都"
HALL_NAME = "exa-first"
EXTRA_WORD = "-データ一覧"
SEARCH_MACHINE = "マイジャグラー"
url = f"https://ana-slo.com/{DATA}/{PREF}/{HALL_NAME}{EXTRA_WORD}/"
print(url)

MAX_RETRIES = 5
DAYS_AGO = 50


https://ana-slo.com/ホールデータ/東京都/exa-first-データ一覧/


In [4]:


for days_ago in range(DAYS_AGO, DAYS_AGO+100):
    
    driver = start_google_chrome(url)
    driver.execute_script(REMOVE_ADS_SCRIPT)
    
    date, hall_name = click_date_link(driver, days_ago, MAX_RETRIES=MAX_RETRIES)
    driver.execute_script(REMOVE_ADS_SCRIPT)

    section_num = click_machine_by_name(driver, SEARCH_MACHINE)

    extract_and_save_model_data(driver, section_num, hall_name, date)
    
    driver.close()
    
    sleep(10)

--------------------------------------------------
https://ana-slo.com/ホールデータ/東京都/exa-first-データ一覧/
クリック対象リンク: 2025/02/04(火)
⚠ Google広告が表示されました。リトライします (1/5)
遷移成功: https://ana-slo.com/2025-02-04-exa-first-data/
日付: 2025-02-04
ホール名: EXA_FIRST
クリック成功: マイジャグラーV
マイジャグラーV: 83 units 取得開始
取得中...
マイジャグラーV: 81 units 取得完了
1001 1,964 +132 7 6 1/151.1 1/280.6 1/327.3
✅ データ保存完了: ../csv/EXA_FIRST_マイジャグラーV_2025-02-04.csv
--------------------------------------------------
https://ana-slo.com/ホールデータ/東京都/exa-first-データ一覧/
クリック対象リンク: 2025/02/03(月)
⚠ Google広告が表示されました。リトライします (1/5)
遷移成功: https://ana-slo.com/2025-02-03-exa-first-data/
日付: 2025-02-03
ホール名: EXA_FIRST
クリック成功: マイジャグラーV
マイジャグラーV: 83 units 取得開始
取得中...
マイジャグラーV: 81 units 取得完了
1001 2,137 -374 7 5 1/178.1 1/305.3 1/427.4
✅ データ保存完了: ../csv/EXA_FIRST_マイジャグラーV_2025-02-03.csv
--------------------------------------------------
https://ana-slo.com/ホールデータ/東京都/exa-first-データ一覧/
クリック対象リンク: 2025/02/02(日)
⚠ Google広告が表示されました。リトライします (1/5)
遷移成功: https://ana-slo.com